# 一. 单例模式（Singleton Pattern）

    是一种常用的软件设计模式，该模式的主要目的是确保某一个类只有一个实例存在。当你希望在整个系统中，某个类只能出现一个实例时，单例对象就能派上用场。
    比如，某个服务器程序的配置信息存放在一个文件中，客户端通过一个 AppConfig 的类来读取配置文件的信息。如果在程序运行期间，有很多地方都需要使用配置文件的内容，也就是说，很多地方都需要创建 AppConfig 对象的实例，这就导致系统中存在多个 AppConfig 的实例对象，而这样会严重浪费内存资源，尤其是在配置文件内容很多的情况下。事实上，类似 AppConfig 这样的类，我们希望在程序运行期间只存在一个实例对象。
    
在 Python 中，我们可以用多种方法来实现单例模式：
1. 使用模块
2. 使用 __new__
3. 使用装饰器（decorator）
4. 使用元类（metaclass）

### 1.1 使用模块
其实，Python 的模块就是天然的单例模式，因为模块在第一次导入时，会生成 .pyc 文件，当第二次导入时，就会直接加载 .pyc 文件，而不会再次执行模块代码。因此，我们只需把相关的函数和数据定义在一个模块中，就可以获得一个单例对象了。

In [10]:
import os
txt ='''
class My_Singleton(object):
    def __init__(self):
        print("create singleton instance")
        self.a = 1
    
my_singleton = My_Singleton()
'''
single_py = 'mysingleton.py'
with open(single_py, 'w') as f:
    f.write(txt)
#-----------------------------------
# other.py
from mysingleton import my_singleton
# key point !!!
sampe = my_singleton

sampe.a = 2
print(my_singleton.a)
os.remove(single_py)

2


### 1.2 使用 __new__

    为了使类只能出现一个实例，我们可以使用 __new__ 来控制实例的创建过程。 
    
    __new__是在实例创建之前被调用的，因为它的任务就是创建实例然后返回该实例，是个静态方法。
    
    __new__在__init__之前被调用，它的第一个值cls 是实例变量对于的类， 然后由super(parent, self)函数返回父类的self
    
    __new__的返回值（实例）将传递给__init__方法的第一个参数，然后__init__给这个实例设置一些参数。
    
    

In [6]:
class Singleton(object):
    _instance = None
    def __init__(self):
        print("Singleton init")
    def __new__(cls, *args, **kw):
        print("Singleton new")
        print('cls',cls)
        if not cls._instance:
            f = super(Singleton, cls)
            print('f',f)
            print(f.__new__)
            cls._instance = f.__new__(cls, *args, **kw)

        return cls._instance
    
class MyClass(Singleton):  
    def __init__(self):
        print("MyClass initialize")
    a = 1
    
one = MyClass()
two = MyClass()
print(one is two)


Singleton new
cls <class '__main__.MyClass'>
f <super: <class 'Singleton'>, <MyClass object>> 94125576
<built-in method __new__ of type object at 0x000000005742C580>
MyClass initialize
Singleton new
cls <class '__main__.MyClass'>
MyClass initialize
True


### 1.3 使用装饰器

    我们知道，装饰器（decorator）可以动态地修改一个类或函数的功能。
    这里，我们也可以使用装饰器来装饰某个类，使其只能生成一个实例，代码如下：

In [21]:
from functools import wraps
def singleton(cls):
    instances = {}
    @wraps(cls)
    def getinstance(*args, **kw):
        if cls not in instances:
            instances[cls] = cls(*args, **kw)
        return instances[cls]
    return getinstance

@singleton
class MyClass(object):
    a = 1
    
one = MyClass()
two = MyClass()
print(one is two)

True


### 1.4 使用 metaclass

    元类（metaclass）可以控制类的创建过程，它主要做三件事：

    1.拦截类的创建
    2.修改类的定义
    3.返回修改后的类
    4.使用元类实现单例模式的代码如下：

In [34]:
class Singleton(type):
    _instances = None
    def __call__(cls, *args, **kwargs):

        if cls._instances is None:
            print(cls)
            cls._instances = super(Singleton, cls).__call__(*args, **kwargs)
        return cls._instances

# Python2
# class MyClass(object):
#     __metaclass__ = Singleton
class MyClass(metaclass=Singleton):
    
    __metaclass__ = Singleton
    print(__metaclass__)

one = MyClass()
two = MyClass()
print(one)
print(two)
print(one is two)

<class '__main__.Singleton'>
<class '__main__.MyClass'>
True


In [ ]:
https://juejin.im/post/58d798160ce463005708c447